# 1. 데이터 불러오기 및 기본 확인

파이썬 3.12 버전을 설치합니다:

In [ ]:
import sys

assert sys.version_info >= (3, 12)

프로젝트 폴더 내 datasets 폴더에서 csv 데이터셋 파일을 가져옵니다:

In [ ]:
# 데이터 불러오기 및 기본 정보 확인
import pandas as pd

# 파일 경로
file_path = "datasets/BostonHousingprice.csv"

# 데이터 읽기
data = pd.read_csv(file_path)

In [ ]:
# 데이터 확인
print(data.head())
print(data.info())
print(data.describe())
print(data.columns)

결측치 확인 및 처리:

In [ ]:
# 결측치 확인
print(data.isnull().sum())

# 결측치 발견시 삭제
data = data.dropna()

이상치 처리:

In [ ]:
import matplotlib.pyplot as plt

# 이상치 확인
plt.boxplot(data['MEDV'])
plt.show()

# 이상치 제거
Q1 = data['MEDV'].quantile(0.25)
Q3 = data['MEDV'].quantile(0.75)
IQR = Q3 - Q1
data = data[(data['MEDV'] >= Q1 - 1.5 * IQR) & (data['MEDV'] <= Q3 + 1.5 * IQR)]

데이터 스케일링:

In [ ]:
from sklearn.preprocessing import StandardScaler

# 주요 설명 변수 스케일링
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[['CRIM', 'B', 'LSTAT']])  # 예: 범죄율, 흑인비율, 저소득층 비율
scaled_data = pd.DataFrame(scaled_features, columns=['CRIM', 'B', 'LSTAT'])


# 2. 탐색적 데이터 분석(Exploratory Data Analysis)

변수 간 상관관계 분석:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 상관계수 히트맵 표시
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

먼저 데이터 내 주택가격에 대한 분포를 확인합니다

In [ ]:
# MEDV(주택 가격) 분포
sns.histplot(data['MEDV'], kde=True, bins=30)
plt.title('Distribution of MEDV')
plt.xlabel('MEDV')
plt.ylabel('Frequency')
plt.show()

MEDV(주택 가격)과 각 데이터간의 상관관계를 분석합니다

In [ ]:
# 변수별 시각화 (Scatterplot)
sns.scatterplot(x=data['B'], y=data['MEDV'])
plt.title("proportion of blacks and MEDV")
plt.xlabel("proportion of blacks (B)")
plt.ylabel("Median Value of Homes (MEDV)")
plt.show()

# RM(방 개수)과 MEDV의 관계
sns.scatterplot(x=data['RM'], y=data['MEDV'])
plt.title('Relationship between RM and MEDV')
plt.xlabel('Number of Rooms (RM)')
plt.ylabel('Median Value of Homes (MEDV)')
plt.show()

# LSTAT(저소득층 비율)과 MEDV의 관계
sns.scatterplot(x=data['LSTAT'], y=data['MEDV'])
plt.title('Relationship between LSTAT and MEDV')
plt.xlabel('% Lower Status of Population (LSTAT)')
plt.ylabel('Median Value of Homes (MEDV)')
plt.show()

In [ ]:
#왜곡 데이터 확인 및 처리
import numpy as np

data['LOG_MEDV'] = np.log1p(data['MEDV'])
sns.histplot(data['LOG_MEDV'], kde=True, bins=30)
plt.title('Log-transformed MEDV Distribution')
plt.show()
